# the `BusyFlyerMixin`

In [1]:
import enum
import logging
import threading
import time

import ophyd
import bluesky
import bluesky.plans
import databroker

logger = logging.getLogger()
RE = bluesky.RunEngine({})
db = databroker.Broker.from_config(databroker.temp_config())
RE.subscribe(db.insert)

0

In [2]:
BUSY_PV = 'prj:mybusy'
TIME_WAVE_PV = 'prj:t_array'
X_WAVE_PV = 'prj:x_array'
Y_WAVE_PV = 'prj:y_array'

In [3]:
class BusyStatus(str, enum.Enum):
    busy = "Busy"
    done = "Done"

class MyWaveform(ophyd.Device):
    """waveform records store fly scan data"""
    wave = ophyd.Component(ophyd.EpicsSignalRO, "")
    number_elements = ophyd.Component(ophyd.EpicsSignalRO, ".NELM")
    number_read = ophyd.Component(ophyd.EpicsSignalRO, ".NORD")

In [4]:
class BusyFlyerDevice(ophyd.Device):
    """
    support APS Fly Scans that are operated by a busy record
    """

    busy = ophyd.Component(ophyd.EpicsSignal, BUSY_PV, string=True)
    time = ophyd.Component(MyWaveform, TIME_WAVE_PV)
    axis = ophyd.Component(MyWaveform, X_WAVE_PV)
    signal = ophyd.Component(MyWaveform, Y_WAVE_PV)
    
    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self.complete_status = None
        self.t0 = time.time()
        self.waves = (self.time, self.axis, self.signal)

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        self.complete_status = ophyd.DeviceStatus(self.busy)
        
        def cb(*args, **kwargs):
            if self.busy.value in (BusyStatus.done):
                self.complete_status._finished(success=True)
        
        self.t0 = time.time()
        self.busy.put(BusyStatus.busy)
        self.busy.subscribe(cb)

        kickoff_status = ophyd.DeviceStatus(self)
        kickoff_status._finished(success=True)
        return kickoff_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete(): " + str(self.complete_status))
        return self.complete_status

    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        schema = {}
        for item in self.waves:
            structure = dict(
                source = item.wave.pvname,
                dtype = "number",
                shape = (1,)
            )
            schema[item.name] = structure
        return {self.name: schema}

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect(): " + str(self.complete_status))
        self.complete_status = None
        for i in range(int(self.time.number_read.value)):
            data = {}
            timestamps = {}
            t = time.time()
            for item in self.waves:
                data[item.name] = item.wave.value[i]
                timestamps[item.name] = t
            data[self.time.name] -= self.t0  # demo: offset time (removes large offset)
            d = dict(
                time=time.time(),
                data=data,
                timestamps=timestamps
            )
            yield d

In [5]:
bflyer = BusyFlyerDevice(name="bflyer")

In [6]:
status = bflyer.complete()
status

In [7]:
list(bflyer.collect())

[{'data': {'bflyer_axis': -1.23,
   'bflyer_signal': 0.16916151674677654,
   'bflyer_time': -101.57950186729431},
  'time': 1524924553.5233662,
  'timestamps': {'bflyer_axis': 1524924553.5190482,
   'bflyer_signal': 1524924553.5190482,
   'bflyer_time': 1524924553.5190482}},
 {'data': {'bflyer_axis': 0.87,
   'bflyer_signal': 0.16916151674677654,
   'bflyer_time': -99.174866914749146},
  'time': 1524924553.525138,
  'timestamps': {'bflyer_axis': 1524924553.5233705,
   'bflyer_signal': 1524924553.5233705,
   'bflyer_time': 1524924553.5233705}},
 {'data': {'bflyer_axis': 2.9700000000000002,
   'bflyer_signal': 0.69233234149691003,
   'bflyer_time': -96.768847942352295},
  'time': 1524924553.5256658,
  'timestamps': {'bflyer_axis': 1524924553.52514,
   'bflyer_signal': 1524924553.52514,
   'bflyer_time': 1524924553.52514}},
 {'data': {'bflyer_axis': 5.0700000000000003,
   'bflyer_signal': 0.521690699626154,
   'bflyer_time': -94.363762855529785},
  'time': 1524924553.5288606,
  'timestamp

In [8]:
import bluesky.utils
RE.msg_hook = bluesky.utils.ts_msg_hook
logging.basicConfig(
    # level=logging.INFO, 
    level=logging.DEBUG, 
    format='%(asctime)s.%(msecs)03d %(levelname)s %(message)s',
    datefmt='%M:%S',
    # datefmt='%Y-%m-%d %H:%M:%S',
    ) 

In [9]:
RE(bluesky.plans.fly([bflyer]))

09:13.594 DEBUG Inserted RunStart with uid d4c7687f-79e4-4d88-abdc-25deb841ad90
09:13.600 INFO kickoff()
09:13.614 INFO complete(): DeviceStatus(device=bflyer_busy, done=False, success=False)


09:09:13.577519 open_run          -> None            args: (), kwargs: {}
09:09:13.600244 kickoff           -> bflyer          args: (), kwargs: {'group': None}
09:09:13.613497 wait              -> None            args: (), kwargs: {'group': None}
09:09:13.614000 complete          -> bflyer          args: (), kwargs: {'group': None}
09:09:13.615034 wait              -> None            args: (), kwargs: {'group': None}


09:31.911 INFO describe_collect()
09:31.934 DEBUG Inserted EventDescriptor with uid 65c6d0b2-b675-4a92-b668-dc14dd04e774 referencing RunStart with uid d4c7687f-79e4-4d88-abdc-25deb841ad90
09:31.936 INFO collect(): DeviceStatus(device=bflyer_busy, done=True, success=True)
09:31.960 DEBUG Inserted RunStop with uid d00032bc-4e94-40d3-bfba-65e6c667f6c2 referencing RunStart  with uid d4c7687f-79e4-4d88-abdc-25deb841ad90


09:09:31.911052 collect           -> bflyer          args: (), kwargs: {'stream': False}
09:09:31.956795 close_run         -> None            args: (), kwargs: {'exit_status': None, 'reason': None}


('d4c7687f-79e4-4d88-abdc-25deb841ad90',)

In [10]:
# RE.abort()

In [11]:
h = db[-1]
h.table("bflyer")

09:31.994 INFO Interpreting key = -1 as an integer


,time,bflyer_time,bflyer_axis,bflyer_signal
seq_num,,,,
1,2018-04-28 10:09:31.937577,8.689862,-1.23,0.005661
2,2018-04-28 10:09:31.941986,11.093268,0.87,0.870741
3,2018-04-28 10:09:31.942594,13.497057,2.97,0.017716
4,2018-04-28 10:09:31.945348,15.900914,5.07,0.350195
5,2018-04-28 10:09:31.945889,18.305213,7.17,0.032456
